In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys
import numpy
from collections import defaultdict
import scipy.sparse as sp
from scipy.sparse.linalg import svds

In [3]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=header)

In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


In [5]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = 1

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = 1

In [24]:
def svd_k(k):
#get SVD components from train matrix. Choose k.
    u, s, vt = svds(train_data_matrix, k = k)
    s_diag_matrix=np.diag(s)
    X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
#print ('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))
    return u, s_diag_matrix, vt

In [28]:
aucs=[]
u5,s5,v5=svd_k(5)
aucs.append(auc(u5,s5,v5,test_data, train_data))
u10,s10,v10=svd_k(10)
aucs.append(auc(u10,s10,v10,test_data, train_data))
u20,s20,v20=svd_k(20)
aucs.append(auc(u20,s20,v20,test_data, train_data))
u30,s30,v30=svd_k(30)
aucs.append(auc(u30,s30,v30,test_data, train_data))
u50,s50,v50=svd_k(50)
aucs.append(auc(u50,s50,v50,test_data, train_data))
u100,s100,v100=svd_k(100)
aucs.append(auc(u100,s100,v100,test_data, train_data))

Current AUC mean (900 samples): 0.92056
Current AUC mean (900 samples): 0.92646
Current AUC mean (900 samples): 0.91603
Current AUC mean (900 samples): 0.90199
Current AUC mean (900 samples): 0.87428
Current AUC mean (900 samples): 0.81552


In [29]:
aucs

[0.92061229244928111,
 0.92649796712607191,
 0.916325895170786,
 0.90238331530429772,
 0.87456191479984624,
 0.81609550841763046]

In [8]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [27]:
    def auc(u,s,v, test_data, train_data):
        """
          Computes the Area Under Curve (AUC) on `test_data`.
          `test_data` is an array of (user_index, item_index) tuples.
          During this computation we ignore users and items
          that didn't appear in the training data, to allow
          for non-overlapping training and testing sets.
        """
        test_dict, test_users, test_items = _data_to_dict(test_data)
        _train_dict, _train_users, _train_items = _data_to_dict(train_data)
        auc_values = []
        z = 0
        for user in test_dict.keys():
            if user in _train_users:
                auc_for_user = 0.0
                n = 0
                predictions = predict_for_user(u,s,v,user-1)
                for pos_item in test_dict[user]:
                    if pos_item in _train_items:
                        for neg_item in _train_items:
                            if neg_item not in test_dict[user] and neg_item not in _train_dict[user]:
                                n += 1
                                #print(user)
                                if predictions[pos_item-1] > predictions[neg_item-1]:
                                    auc_for_user += 1
                if n > 0:
                    auc_for_user /= n
                    auc_values.append(auc_for_user)
                z += 1
                if z % 100 == 0 and len(auc_values) > 0:
                    sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z), numpy.mean(auc_values)))
                    sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [11]:
def predict_for_user(u,s_diag_matrix, vt, n):
    return np.dot(np.dot(u[n], s_diag_matrix), vt)

In [12]:
    def _data_to_dict(data):
        data_dict = defaultdict(list)
        items = set()
        for user in data.iterrows():
            data_dict[int(user[1][0])].append(int(user[1][1]))
            items.add(int(user[1][1]))
        return data_dict, set(data_dict.keys()), items